In [0]:
from pyspark.sql.types import (
    StructType, StructField,
    StringType, IntegerType, DoubleType, TimestampType
)

bronze_sales_schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("transaction_timestamp", TimestampType(), True),  
    StructField("store_id", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("unit_price", IntegerType(), True),   
    StructField("discount", IntegerType(), True),
    StructField("total_amount", IntegerType(), True),
    StructField("currency", StringType(), True)
])


In [0]:
# ============================
# BRONZE SALES INGESTION
# ============================

from pyspark.sql.functions import current_timestamp, lit, col
from pyspark.sql.types import (
    StructType, StructField,
    StringType, TimestampType, LongType
)
from datetime import datetime

# ----------------------------
# CONFIG
# ----------------------------
raw_sales_path = "dbfs:/Volumes/workspace/retail/raw/bronze/sales/"
bronze_delta_path = "dbfs:/Volumes/workspace/retail/raw/bronze_delta/sales"
source_system = "retail_sales_csv"

pipeline_name = "bronze_sales_ingestion"
layer = "bronze"
start_time = datetime.now()

# ----------------------------
# LOG SCHEMA (REUSED)
# ----------------------------
log_schema = StructType([
    StructField("pipeline_name", StringType(), True),
    StructField("layer", StringType(), True),
    StructField("start_time", TimestampType(), True),
    StructField("end_time", TimestampType(), True),
    StructField("status", StringType(), True),
    StructField("records_processed", LongType(), True),
    StructField("records_rejected", LongType(), True),
    StructField("error_message", StringType(), True)
])

def log_pipeline_event(
    pipeline_name,
    layer,
    start_time,
    end_time,
    status,
    records_processed,
    records_rejected,
    error_message=None
):
    log_data = [(
        pipeline_name,
        layer,
        start_time,
        end_time,
        status,
        int(records_processed),
        int(records_rejected),
        error_message if error_message else ""
    )]

    log_df = spark.createDataFrame(log_data, schema=log_schema)

    (
        log_df.write
        .format("delta")
        .mode("append")
        .saveAsTable("workspace.retail.pipeline_logs")
    )

# ----------------------------
# INGESTION WITH LOGGING
# ----------------------------
try:
    # 1 Read raw CSV files
    raw_df = (
        spark.read
        .option("header", True)
        .schema(bronze_sales_schema)
        .csv(raw_sales_path)
    )

    # 2 Add ingestion metadata
    bronze_df = (
        raw_df
        .withColumn("ingestion_timestamp", current_timestamp())
        .withColumn("source_system", lit(source_system))
        .withColumn("source_file", col("_metadata.file_path"))
    )

    record_count = bronze_df.count()

    # 3 Write to Bronze Delta (append-safe)
    (
        bronze_df.write
        .format("delta")
        .mode("append")
        .save(bronze_delta_path)
    )

    end_time = datetime.now()

    # 4 SUCCESS LOG
    log_pipeline_event(
        pipeline_name,
        layer,
        start_time,
        end_time,
        status="SUCCESS",
        records_processed=record_count,
        records_rejected=0
    )

    print(f"Bronze ingestion completed successfully. Records ingested: {record_count}")

except Exception as e:
    end_time = datetime.now()

    # 5 FAILURE LOG
    log_pipeline_event(
        pipeline_name,
        layer,
        start_time,
        end_time,
        status="FAILED",
        records_processed=0,
        records_rejected=0,
        error_message=str(e)
    )

    print("Bronze ingestion failed")
    raise


Bronze ingestion completed successfully. Records ingested: 247


In [0]:
%sql
SELECT *
FROM workspace.retail.pipeline_logs
WHERE layer = 'bronze'
ORDER BY start_time DESC;


pipeline_name layer start_time end_time status records_processed records_rejected error_message bronze_sales_ingestion bronze 2025-12-31T13:58:23.875Z 2025-12-31T13:58:26.795Z SUCCESS 247 0 bronze_sales_ingestion bronze 2025-12-31T13:32:05.959Z 2025-12-31T13:32:08.142Z SUCCESS 247 0 bronze_sales_ingestion bronze 2025-12-31T13:27:26.733Z 2025-12-31T13:27:29.592Z SUCCESS 247 0 bronze_sales_ingestion bronze 2025-12-31T13:25:56.056Z 2025-12-31T13:25:58.174Z FAILED 0 0 [DELTA_FAILED_TO_MERGE_FIELDS] Failed to merge fields 'unit_price' and 'unit_price'.

JVM stacktrace:
com.databricks.sql.transaction.tahoe.DeltaAnalysisException
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.$anonfun$mergeDataTypes$1(SchemaMergingUtils.scala:231)
	at scala.collection.ArrayOps$.map$extension(ArrayOps.scala:936)
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.merge$1(SchemaMergingUtils.scala:217)
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.mergeDataTypes(SchemaMergingUtils.scala:335)
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.mergeSchemas(SchemaMergingUtils.scala:179)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation$.mergeSchema(ImplicitMetadataOperation.scala:359)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation.updateMetadata(ImplicitMetadataOperation.scala:112)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation.updateMetadata$(ImplicitMetadataOperation.scala:92)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.updateMetadata(WriteIntoDeltaEdge.scala:120)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.writeAndReturnCommitDataAndMaterializationPlans(WriteIntoDeltaEdge.scala:411)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.writeAndReturnCommitData(WriteIntoDeltaEdge.scala:256)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.$anonfun$run$2(WriteIntoDeltaEdge.scala:182)
	at com.databricks.sql.transaction.tahoe.DeltaLog.withNewTransaction(DeltaLog.scala:325)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.$anonfun$run$1(WriteIntoDeltaEdge.scala:169)
	at com.databricks.sql.acl.CheckPermissions$.$anonfun$trusted$2(CheckPermissions.scala:2642)
	at com.databricks.sql.util.ThreadLocalTagger.withTag(QueryTagger.scala:63)
	at com.databricks.sql.util.ThreadLocalTagger.withTag$(QueryTagger.scala:60)
	at com.databricks.sql.util.QueryTagger$.withTag(QueryTagger.scala:235)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:2642)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.run(WriteIntoDeltaEdge.scala:168)
	at com.databricks.sql.transaction.tahoe.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:356)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:57)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.$anonfun$sideEffectResult$2(commands.scala:87)
	at org.apache.spark.sql.execution.SparkPlan.runCommandInAetherOrSpark(SparkPlan.scala:195)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.$anonfun$sideEffectResult$1(commands.scala:87)
	at com.databricks.spark.util.FrameProfiler$.$anonfun$record$1(FrameProfiler.scala:114)
	at com.databricks.spark.util.FrameProfilerExporter$.maybeExportFrameProfiler(FrameProfilerExporter.scala:200)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:105)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:84)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:83)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:97)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$eagerlyExecuteCommands$5(QueryExecution.scala:596)
	at com.databricks.util.LexicalThreadLocal$Ha